# Fix label file

Fix bug where compressed videos sometimes self referenced (target == name)

In [1]:
import pandas as pd
import os
from pathlib import Path

main_df = pd.read_csv(Path('D:/collated/labels_with_original_path.csv'))

In [3]:
same_df = main_df[main_df['target'] == main_df['name']]
same_df

,name,target,label,technique,source,compression,augmentation,tool,content,original_path
2,0002.mp4,0002.mp4,1,faceswap,DFD,c40,-,-,-,D:\datasets\ff-videos\manipulated_sequences\De...
7,0007.mp4,0007.mp4,1,faceswap,DFDC,c40,-,-,-,D:\datasets\DFDC\0\fhcccydsuf.mp4
22,0022.mp4,0022.mp4,1,faceswap,CelebDF,c23,-,-,-,D:\datasets\Celeb-DF-v2\Celeb-synthesis\id50_i...
48,0048.mp4,0048.mp4,1,faceswap,CelebDF,c40,-,-,-,D:\datasets\Celeb-DF-v2\Celeb-synthesis\id12_i...
66,0066.mp4,0066.mp4,1,lipsync,FakeAVCeleb,c23,-,-,-,D:\datasets\FakeAVCeleb_v1.2\FakeVideo-RealAud...
...,...,...,...,...,...,...,...,...,...,...
3739,3739.mp4,3739.mp4,1,faceswap,CelebDF,c40,-,-,-,D:\datasets\Celeb-DF-v2\Celeb-synthesis\id4_id...
3741,3741.mp4,3741.mp4,1,faceswap,DFDC,c40,-,-,-,D:\datasets\DFDC\1\dmfanmmpwx.mp4
3752,3752.mp4,3752.mp4,1,faceswap,CelebDF,c40,-,-,-,D:\datasets\Celeb-DF-v2\Celeb-synthesis\id32_i...
3757,3757.mp4,3757.mp4,1,faceswap,FakeAVCeleb,c40,-,-,-,D:\datasets\FakeAVCeleb_v1.2\FakeVideo-RealAud...


In [11]:
for index, row in same_df.iterrows():
    indexes = main_df.index[main_df['original_path'] == row['original_path']].tolist()
    new_target = f'{[i for i in indexes if i != index][0]}.mp4'
    main_df.at[index, 'target'] = new_target
    

In [12]:
same_df = main_df[main_df['target'] == main_df['name']]
same_df

,name,target,label,technique,source,compression,augmentation,tool,content,original_path


In [13]:
main_df.to_csv(os.path.join(Path('D:/collated'), 'labels.csv'), index=False,
                                        columns=['name', 'target', 'label', 'technique', 'source', 'compression', 'augmentation', 'tool', 'content'])
    
main_df.to_csv(os.path.join(Path('D:/collated'), 'labels_with_original_path.csv'), index=False,
                                        columns=['name', 'target', 'label', 'technique', 'source', 'compression', 'augmentation', 'tool', 'content', 'original_path'])


## Add missing 80 files from 4 missing techniques (Celeb-DF and CelebAV)

In [27]:
sub_df = main_df[(main_df['source'] != 'SelfGenerated') & (main_df['original_path'].str.contains('ncs_3')) & (main_df['compression'] == 'original')].drop_duplicates(subset='target')


In [34]:
import numpy as np

augmentation_techniques = ['vmifgsm_3', 'vmifgsm_6', 'decowa_3', 'decowa_6']
last_index = 3786
new_df = sub_df.loc[sub_df.index.repeat(len(augmentation_techniques))].reset_index(drop=True)
new_df['augmentation'] = np.tile(augmentation_techniques, len(sub_df))
for index, row in new_df.iterrows():
    new_df.at[index, 'name'] = f'{last_index}.mp4'
    last_index += 1
    new_df.at[index, 'original_path'] = row['original_path'].replace('ncs_3', row['augmentation'])
new_df


,name,target,label,technique,source,compression,augmentation,tool,content,original_path
0,3786.mp4,2174.mp4,1,faceswap,CelebDF,original,vmifgsm_3,-,-,D:\datasets\augmented\id51_id53_0004_vmifgsm_3...
1,3787.mp4,2174.mp4,1,faceswap,CelebDF,original,vmifgsm_6,-,-,D:\datasets\augmented\id51_id53_0004_vmifgsm_6...
2,3788.mp4,2174.mp4,1,faceswap,CelebDF,original,decowa_3,-,-,D:\datasets\augmented\id51_id53_0004_decowa_3.mp4
3,3789.mp4,2174.mp4,1,faceswap,CelebDF,original,decowa_6,-,-,D:\datasets\augmented\id51_id53_0004_decowa_6.mp4
4,3790.mp4,0912.mp4,1,faceswap,CelebDF,original,vmifgsm_3,-,-,D:\datasets\augmented\id48_id40_0007_vmifgsm_3...
5,3791.mp4,0912.mp4,1,faceswap,CelebDF,original,vmifgsm_6,-,-,D:\datasets\augmented\id48_id40_0007_vmifgsm_6...
6,3792.mp4,0912.mp4,1,faceswap,CelebDF,original,decowa_3,-,-,D:\datasets\augmented\id48_id40_0007_decowa_3.mp4
7,3793.mp4,0912.mp4,1,faceswap,CelebDF,original,decowa_6,-,-,D:\datasets\augmented\id48_id40_0007_decowa_6.mp4
8,3794.mp4,2801.mp4,1,faceswap-lipsync-voiceclone,FakeAVCeleb,original,vmifgsm_3,-,-,D:\datasets\augmented\00109_7_id01452_wavtolip...
9,3795.mp4,2801.mp4,1,faceswap-lipsync-voiceclone,FakeAVCeleb,original,vmifgsm_6,-,-,D:\datasets\augmented\00109_7_id01452_wavtolip...


In [35]:
import random

compressed_df = new_df.copy()
compressed_df['target'] = compressed_df['name']
indices = random.sample(range(len(compressed_df)), len(compressed_df)//2)
compressed_df['compression'] = 'c23'
compressed_df.loc[indices,'compression'] = 'c40'

for index, row in compressed_df.iterrows():
    compressed_df.at[index, 'name'] = f'{last_index}.mp4'
    last_index += 1

combined_df = pd.concat([new_df, compressed_df], ignore_index=True)
combined_df

,name,target,label,technique,source,compression,augmentation,tool,content,original_path
0,3786.mp4,2174.mp4,1,faceswap,CelebDF,original,vmifgsm_3,-,-,D:\datasets\augmented\id51_id53_0004_vmifgsm_3...
1,3787.mp4,2174.mp4,1,faceswap,CelebDF,original,vmifgsm_6,-,-,D:\datasets\augmented\id51_id53_0004_vmifgsm_6...
2,3788.mp4,2174.mp4,1,faceswap,CelebDF,original,decowa_3,-,-,D:\datasets\augmented\id51_id53_0004_decowa_3.mp4
3,3789.mp4,2174.mp4,1,faceswap,CelebDF,original,decowa_6,-,-,D:\datasets\augmented\id51_id53_0004_decowa_6.mp4
4,3790.mp4,0912.mp4,1,faceswap,CelebDF,original,vmifgsm_3,-,-,D:\datasets\augmented\id48_id40_0007_vmifgsm_3...
...,...,...,...,...,...,...,...,...,...,...
75,3861.mp4,3821.mp4,1,faceswap,CelebDF,c23,decowa_6,-,-,D:\datasets\augmented\id1_id6_0003_decowa_6.mp4
76,3862.mp4,3822.mp4,1,faceswap,CelebDF,c23,vmifgsm_3,-,-,D:\datasets\augmented\id6_id0_0008_vmifgsm_3.mp4
77,3863.mp4,3823.mp4,1,faceswap,CelebDF,c40,vmifgsm_6,-,-,D:\datasets\augmented\id6_id0_0008_vmifgsm_6.mp4
78,3864.mp4,3824.mp4,1,faceswap,CelebDF,c23,decowa_3,-,-,D:\datasets\augmented\id6_id0_0008_decowa_3.mp4


In [36]:
import shutil

output_dir = Path('D:/addition')
os.makedirs(output_dir, exist_ok=True)
ffmpeg_compress = 'ffmpeg -i "{input_video_file}" -vcodec libx264 -y -crf {rate_factor} {output_video_file}'

for _, row in combined_df.iterrows():
    output_video_file = output_dir / row['name']
    if row['compression'] == 'c23':
        os.system(ffmpeg_compress.format(input_video_file = row['original_path'], output_video_file = output_video_file, rate_factor = 23))
    elif row['compression'] == 'c40':
        os.system(ffmpeg_compress.format(input_video_file = row['original_path'], output_video_file = output_video_file, rate_factor = 40))
    else:
        shutil.copy(row['original_path'], output_video_file)

In [37]:
combined_df = pd.concat([main_df, combined_df])
combined_df.to_csv(os.path.join(output_dir, 'labels.csv'), index=False,
                                        columns=['name', 'target', 'label', 'technique', 'source', 'compression', 'augmentation', 'tool', 'content'])
    
combined_df.to_csv(os.path.join(output_dir, 'labels_with_original_path.csv'), index=False,
                                        columns=['name', 'target', 'label', 'technique', 'source', 'compression', 'augmentation', 'tool', 'content', 'original_path'])
